In [26]:
import math
import numpy as np
import pandas as pd

In [27]:
ent = pd.read_csv("entries.csv", encoding="SHIFT-JIS")

In [28]:
ent

,TDO_Rank,Name,Fast Move,Charge Move,SCP,Type1,Type2
0,1,ピジョット,かぜおこし,ブレイブバード,1559,ノーマル,ひこう
1,2,ゴローニャ(アローラ),ボルトチェンジ,ワイルドボルト,1487,いわ,でんき
2,3,キュウコン,ほのおのうず,オーバーヒート,1575,ほのお,-
3,4,フシギバナ,つるのムチ,ハードプラント,1508,くさ,どく
4,5,スリーパー,ねんりき,みらいよち,1695,エスパー,-
...,...,...,...,...,...,...,...
294,295,サイドン,どろかけ,じしん,1477,いわ,じめん
295,296,ウツドン,タネマシンガン,パワーウィップ,1383,くさ,どく
296,297,エレブー,でんきショック,かみなり,1431,でんき,-
297,298,ハクリュー,りゅうのいぶき,りゅうのはどう,1486,ドラゴン,-


In [29]:
na = pd.read_csv("normal_attacks.csv", encoding="SHIFT-JIS", index_col=0).T

In [30]:
na["ロックオン"]

威力          1
ゲージ量        5
時間(T)       1
DPT         1
EPT         5
Type1    ノーマル
Name: ロックオン, dtype: object

In [31]:
na["ロックオン"]["EPT"]

5.0

In [32]:
sa = pd.read_csv("special_attacks.csv", encoding="SHIFT-JIS", index_col=0).T

In [33]:
sa["シャドーボーン"]["DPE"]

1.78

In [34]:
pt = pd.read_csv("poke_type.csv", encoding="SHIFT-JIS", index_col=0)

In [35]:
pt

,ノーマル,ほのお,みず,くさ,でんき,こおり,かくとう,どく,じめん,ひこう,エスパー,むし,いわ,ゴースト,ドラゴン,あく,はがね,フェアリー,SUM
Atack/Defence,,,,,,,,,,,,,,,,,,,
SUM,-1,-3,-2,1,-1,3,0,-3,-1,-2,1,2,1,-2,-1,-1,-9,-3,NaN
かくとう,1,0,0,0,0,1,0,-1,0,-1,-1,1,1,0,0,1,1,-1,2.0
こおり,0,-1,-1,1,1,-1,0,0,1,1,0,0,0,0,1,0,-1,0,1.0
じめん,0,1,0,-1,1,0,0,1,0,-2,0,-1,1,0,0,0,1,0,1.0
いわ,0,1,0,0,0,1,-1,0,-1,1,0,1,0,0,0,0,-1,0,1.0
ほのお,0,-1,-1,1,0,1,0,0,0,0,0,1,-1,0,-1,0,1,0,0.0
みず,0,1,-1,-1,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0.0
ひこう,0,0,0,1,-1,0,1,0,0,0,0,1,-1,0,0,0,-1,0,0.0
フェアリー,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,1,1,-1,0,0.0


In [36]:
class Pokemon:
    def __init__(self, name, fast, charge, scp, type1, type2):
        self.name = name
        self.fast = fast
        self.charge = charge
        self.scp = scp
        self.type1 = type1
        self.type2 = type2
        
    def PCP(self, enemy):
        return math.pow(self.scp * 10, 3/2) * self.DPT_PVP(enemy) * 10 ** (-4)
    
    def DPT_PVP(self, enemy):
        fastDamage = na[self.fast]["DPT"] * self.DamageFactor(na[self.fast]["Type1"], enemy)
        chargeDamage = na[self.fast]["EPT"] * sa[self.charge]["DPE"] * self.DamageFactor(sa[self.charge]["Type1"], enemy)
        return fastDamage + chargeDamage
    
    def DamageFactor(self, moveType, enemy):
        return self.STF(moveType) * self.ME(moveType, enemy)
    
    def STF(self, moveType):
        if (moveType == self.type1 or moveType == self.type2):
            return 1.2
        else:
            return 1.0
        
    def ME(self, moveType, enemy):
        ex = pt[enemy.type1][moveType]
        
        if (enemy.type2 != '-'):
            ex += pt[enemy.type2][moveType]
            
        return math.pow(1.6, ex)

In [37]:
class PCP_Row:
    def __init__(self, pokemon):
        self.pokemon = pokemon
        self.total_pcp = 0
        self.total_opponent_pcp = 0
    
    def calc(self, enemy):
        self.total_pcp += self.pokemon.PCP(enemy)
        self.total_opponent_pcp += enemy.PCP(self.pokemon)
        
    def row(self, num):
        apcp = self.total_pcp / num
        oapcp = self.total_opponent_pcp / num
        rate = apcp / oapcp
        return (self.pokemon.name, self.pokemon.fast, self.pokemon.charge, self.pokemon.scp, self.pokemon.type1, self.pokemon.type2, apcp, oapcp, rate)

In [38]:
pokemons = [Pokemon(row["Name"], row["Fast Move"], row["Charge Move"], row["SCP"], row["Type1"], row["Type2"]) for idx, row in ent.iterrows()]

In [39]:
rows = [PCP_Row(poke) for poke in pokemons]

In [40]:
for r in rows:
    for poke in pokemons:
        r.calc(poke)

In [41]:
table = [r.row(len(rows)) for r in rows]

In [42]:
df = pd.DataFrame(table, columns=["Name", "FastMove", "ChargeMove", "SCP", "Type1", "Type2", "APCP", "OAPCP", "APCP/OAPCP"])

In [43]:
df.sort_values("APCP/OAPCP", ascending=False, inplace=True)
df

,Name,FastMove,ChargeMove,SCP,Type1,Type2,APCP,OAPCP,APCP/OAPCP
2,キュウコン,ほのおのうず,オーバーヒート,1575,ほのお,-,2654.855940,1938.876094,1.369276
22,ガラガラ(アローラ),たたりめ,シャドーボール,1625,ほのお,ゴースト,2407.586169,1803.594404,1.334882
31,ガラガラ(アローラ),たたりめ,シャドーボーン,1625,ほのお,ゴースト,2366.076063,1803.594404,1.311867
33,ガラガラ(アローラ),ほのおのうず,シャドーボール,1625,ほのお,ゴースト,2341.020339,1803.594404,1.297975
3,フシギバナ,つるのムチ,ハードプラント,1508,くさ,どく,2592.899934,2019.779950,1.283754
...,...,...,...,...,...,...,...,...,...
237,ラプラス,みずでっぽう,ロケットずつき,1675,みず,こおり,1887.952123,2253.472343,0.837797
293,ラプラス,こおりのいぶき,ロケットずつき,1675,みず,こおり,1883.296781,2253.472343,0.835731
279,パルシェン,こおりのつぶて,おんがえし,1538,みず,こおり,1873.939506,2253.472343,0.831579
263,キュウコン(アローラ),あまえる,サイコショック,1575,こおり,フェアリー,1797.176157,2182.268371,0.823536


In [44]:
df.to_csv("APCP_OAPCP_Ranking.csv", encoding="SHIFT-JIS", index=False)